In [67]:
import os
import re
import spacy
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [75]:
def get_surah_names():
    surah_names = [] #surah names sorted
    URL = "https://surahquran.com/quran-search/quran.html"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    all_table = soup.find_all('table')[1]
    for idx, elm in enumerate(all_table.find_all("a")):
        surah_names.append(
            (idx + 1, elm.text)
        )
    return surah_names

In [77]:
book = pd.read_csv(
    f'{os.getcwd()}/data/french.csv',
    sep="|",
    header=None,
)
book.columns = ["sura", "ayat", "text"]
book['sura'] = book['sura'].astype(int)
book['ayat'] = book['ayat'].astype(int)

mapping_dictionary = {t[0]: t for t in get_surah_names()}
book['sura_name'] = book['sura'].map(mapping_dictionary).apply(lambda t: t[1])

def cleaning_text(text):
    text = re.sub(r'` ', "'", text)
    text = re.sub(r' +(\.)', r'\1', text)
    return text

book["text"] = book["text"].apply(lambda x: cleaning_text(x))
book['text'].to_csv(f'{os.getcwd()}/output.txt', index=False, sep="\t", header=None)

In [60]:
nlp = spacy.load('fr_core_news_sm')

with open(f'{os.getcwd()}/output.txt') as file:
    text = file.read().replace('\n', ' ')

def find_sentences():
    doc = nlp(text)
    sentences = list(doc.sents)
    return sentences

In [161]:
from thefuzz import fuzz

search_str = "par un tel exemple"
book = book.assign(
    score = lambda _df: 
        _df['text'].map(lambda t: 
            fuzz.partial_ratio(t, search_str)
        )
    )

In [ ]:
book.sort_values('score', ascending=False)
book.loc[32, 'text']